In [142]:
# !pip install torch --quiet
# !pip uninstall torchtext -y
# !pip install torchtext==0.9.0 --quiet

In [143]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(sequential = True, tokenize = 'spacy',batch_first =True,
                  tokenizer_language = 'en_core_web_sm', include_lengths=True) 
LABEL = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)  # dtype = torch.float, 

In [144]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [145]:
train_data.data

<generator object Dataset.__getattr__ at 0x7f7b1fa87c80>

In [147]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
# train_data = train_data_updated
# test_data = test_data_updated

Number of training examples: 25000
Number of testing examples: 25000


In [149]:
# print(vars(train_data.examples[0]))

In [150]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [151]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [152]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [153]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [154]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203651), (',', 193608), ('.', 166336), ('and', 109998), ('a', 109919), ('of', 101338), ('to', 94267), ('is', 76650), ('in', 61487), ('I', 54167), ('it', 53630), ('that', 49480), ('"', 44446), ("'s", 43722), ('this', 42462), ('-', 37643), ('/><br', 35683), ('was', 35148), ('as', 30278), ('with', 30114)]


In [155]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [156]:
print(LABEL.vocab.stoi)
print('Labels : ', TEXT.vocab.stoi)


defaultdict(None, {'neg': 0, 'pos': 1})
Labels :  defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f7b04c11e80>>, {'<unk>': 0, '<pad>': 1, 'the': 2, ',': 3, '.': 4, 'and': 5, 'a': 6, 'of': 7, 'to': 8, 'is': 9, 'in': 10, 'I': 11, 'it': 12, 'that': 13, '"': 14, "'s": 15, 'this': 16, '-': 17, '/><br': 18, 'was': 19, 'as': 20, 'with': 21, 'movie': 22, 'for': 23, 'film': 24, 'The': 25, 'but': 26, '(': 27, 'on': 28, "n't": 29, ')': 30, 'you': 31, 'are': 32, 'not': 33, 'have': 34, 'his': 35, 'be': 36, 'he': 37, 'one': 38, '!': 39, 'by': 40, 'at': 41, 'all': 42, 'an': 43, 'who': 44, 'from': 45, 'they': 46, 'like': 47, 'so': 48, 'her': 49, 'or': 50, 'about': 51, 'has': 52, 'just': 53, 'out': 54, 'It': 55, "'": 56, 'do': 57, '?': 58, 'some': 59, 'good': 60, 'more': 61, 'would': 62, 'very': 63, 'up': 64, 'This': 65, 'what': 66, 'there': 67, 'time': 68, 'can': 69, 'which': 70, 'when': 71, 'she': 72, 'had': 73, 'really': 74, 'story': 75, 'only': 76, 'if': 77

In [157]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), sort_key = lambda x: len(x.text),
                                                            sort_within_batch=True,
    batch_size = BATCH_SIZE,
    device = device)
print(type(train_iterator))

cpu
<class 'torchtext.legacy.data.iterator.BucketIterator'>


In [158]:
print(len(train_iterator))
print(len(next(iter(train_iterator))))
     

274
64


In [159]:
import torch.nn as nn
import sys
import sys
import torch.nn.functional as F
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        # self.encoder = nn.RNN(embedding_dim, hidden_dim) # RNN 
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=1, batch_first=True)  # LSTM
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths ):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        # print('Step 2', embedded.shape) 

        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        # print('Step 3', packed_embedded.data.shape)  # [476, 300])  # 476 total words in 32 sentences and each word has 300 dim embedding. This excludes <pad> and <unk> sequences in input
        # output, hidden = self.encoder(embedded)  # RNN
        output, (hidden, cell) = self.encoder(packed_embedded) # LSTM
        # print('Step 4', hidden.shape) # torch.Size([2, 32, 100]) - 2 layers of LSTM with 32 output words (output of LSTM is a word not a sentence) and 100 as hidden layer dimension        
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        dense_outputs = self.fc(hidden)
        # print('step 5', dense_outputs.shape)

        output = F.sigmoid(dense_outputs[0])
        # print('Step 6', output.shape)  # , output)
        # sys.exit()
        # assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return output  # self.fc(hidden.squeeze(0))

In [160]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 20
HIDDEN_DIM = 5
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [161]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 500,586 trainable parameters


In [162]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [163]:
# m = nn.Sigmoid()
# loss = nn.BCELoss()
# input = torch.randn(3, requires_grad=True)
# target = torch.empty(3).random_(2)
# print(m(input).dtype, target.dtype)
# output = loss(m(input), target)
# # output.backward()

In [164]:
criterion = nn.BCELoss()
# criterion = nn.BCEWithLogitsLoss()  # 

In [165]:
model = model.to(device)
criterion = criterion.to(device)

In [166]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [167]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()

        # print(type(batch.text), (batch.text.shape) )  # torch.Size([1016, 64])
        # print(len(batch.text), (batch.text)[1] , (batch.text)[0].shape, type(batch.text))  # , batch.text.shape)
        text, text_lengths = batch.text 
                
        predictions = model(text, text_lengths).squeeze(1)
        batch_labels = (batch.label).to(torch.float32)
        # print(predictions.shape, batch_labels.dtype, predictions.dtype, 'vijsid')  # torch.Size([64])
        # sys.exit()
        loss = criterion(predictions, batch_labels)
        
        acc = binary_accuracy(predictions, batch_labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [168]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            text, text_lengths = batch.text 
            
            # convert to 1d tensor
            predictions = model(text, text_lengths).squeeze(1)
            batch_labels = (batch.label).to(torch.float32)
            # predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch_labels)
            
            acc = binary_accuracy(predictions, batch_labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [169]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [170]:
N_EPOCHS = 2

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

# print(len(batch.text), (batch.text)[1] , (batch.text)[0].shape, type(batch.text))

Epoch: 01 | Epoch Time: 2m 0s
	Train Loss: 0.697 | Train Acc: 49.64%
	 Val. Loss: 0.698 |  Val. Acc: 50.79%
Epoch: 02 | Epoch Time: 1m 58s
	Train Loss: 0.697 | Train Acc: 49.67%
	 Val. Loss: 0.697 |  Val. Acc: 50.79%


In [171]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.697 | Test Acc: 50.01%
